# Shift

Add parent directory to `sys.path` so that we can import `ae483tools`.

In [ ]:
import sys, os
sys.path.append(os.path.abspath('..'))

Do all other imports.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ae483tools import *

Load data from a flight in which the drone took off, moved in a square, and landed.

In [ ]:
raw_data_drone, raw_data_mocap, raw_data_bodies = load_hardware_data('square.json')

Resample drone data.

In [ ]:
data_drone = resample_data_drone(raw_data_drone, t_min_offset=0., t_max_offset=0.)

t = data_drone['time']
x_drone = data_drone['stateEstimate.x']
y_drone = data_drone['stateEstimate.y']
z_drone = data_drone['stateEstimate.z']
psi_drone = np.deg2rad(data_drone['stateEstimate.yaw'])
theta_drone = - np.deg2rad(data_drone['stateEstimate.pitch'])
phi_drone = np.deg2rad(data_drone['stateEstimate.roll'])

Resample mocap data **with a non-zero time shift** and apply coordinate transformation.

In [ ]:
resampled_data_mocap = resample_data_mocap(raw_data_mocap, t, t_shift=1.)
transformed_data_mocap = transform_data_mocap(resampled_data_mocap)

# Only get z estimate from mocap for now
z_mocap = transformed_data_mocap['z']

Plot data.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3), tight_layout=True)
ax.plot(t, z_drone, label='z (drone)')
ax.plot(t, z_mocap, '--', label='z (mocap)')
ax.legend()
ax.grid()

Define parameters to use when searching for a time shift that aligns mocap data with drone data.

In [ ]:
# Choose whether to apply the transform that aligns mocap data in space
do_transform = True

# Choose search interval
t_shift_bnds = [0., 1.]

# Choose resolution
t_shift_res = 0.05

# Choose how many seconds to ignore (i.e., delete) at the start of the mocap data
t_min_offset = 0.

Find the root-mean-square-error (RMSE) for a range of time shifts.

In [ ]:
# Create an array with a range of time shifts
t_shifts = np.linspace(
    t_shift_bnds[0],
    t_shift_bnds[1],
    int(1 + np.ceil((t_shift_bnds[1] - t_shift_bnds[0]) / t_shift_res)),
)

# Print a warning if computation time is likely to be long
if len(t_shifts) > 50:
    print(f'========================================================================')
    print(f'WARNING (sync_data_mocap)\n')
    print(f'Searching the interval\n')
    print(f'  t_shift_bnds=[{min(t_shifts):4.2f}, {max(t_shifts):4.2f}]\n')
    print(f'with the resolution\n')
    print(f'  t_shift_res={t_shift_res:4.2f}\n')
    print(f'means computing RMSE for {len(t_shifts)} possible time shifts, which is likely')
    print(f'to be slow. You may want to decrease the size of the search interval')
    print(f'or increase the resolution.')
    print(f'========================================================================\n')

# Create an array to hold the RMSE for each time shift
RMSEs = np.empty_like(t_shifts)

# Find the RMSE for each time shift
for i, t_shift in enumerate(t_shifts):
    # Resample mocap data with time shift
    resampled_data_mocap = resample_data_mocap(
        raw_data_mocap,
        t,
        t_shift=t_shift,
        t_min_offset=t_min_offset,
    )

    # Transform mocap data
    if do_transform:
        transformed_data_mocap = transform_data_mocap(resampled_data_mocap)
    else:
        transformed_data_mocap = resampled_data_mocap

    # Get z estimate from mocap data
    z_mocap = transformed_data_mocap['z']

    # Find RMSE between z_mocap and z_drone
    RMSEs[i] = np.sqrt(np.mean((z_mocap - z_drone)**2))

Plot the RMSE for a range of time shifts.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3), tight_layout=True)
ax.plot(t_shifts, RMSEs)
ax.grid()

Find the time shift that gives the minimum RMSE.

(Please do **NOT** find `i_min`, `RMSE_min`, and `t_shift_min` by inspection from the plots. Instead, write code that finds each of these quantities in an automated way, so that you can use it for any data you collect, not just for the data you are looking at in this example.)

In [ ]:
# Find the index of the minimum RMSE
i_min = 0           # <-- FIXME

# Find the minimum RMSE
RMSE_min = 0.       # <-- FIXME

# Find the time shift that gives the minimum RMSE
t_shift_min = 0.    # <-- FIXME

Plot the RMSE for a range of time shifts again, this time showing the minimum.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3), tight_layout=True)
ax.plot(t_shifts, RMSEs)
ax.plot(t_shift_min, RMSE_min, '.', markersize=12)
ax.grid()

Define a function to synchronize mocap data with drone data (i.e., to find the time shift that best aligns transformed mocap data with drone data).

In [ ]:
def sync_data_mocap(raw_data_mocap, t, z_drone,
                    do_transform=True, t_min_offset=0.,
                    t_shift_bnds=[0., 1.], t_shift_res=0.05):
    # Find the time shift the minimizes RMSE
    #
    # ...
    # FIXME (your code here)
    # ...
    #
    t_shift_min = 0.    # <-- FIXME

    # Resample mocap data with the time shift that minimizes RMSE
    resampled_data_mocap = resample_data_mocap(
        raw_data_mocap,
        t,
        t_shift=t_shift_min,
        t_min_offset=t_min_offset,
    )
    
    # Transform mocap data
    if do_transform:
        transformed_data_mocap = transform_data_mocap(resampled_data_mocap)
    else:
        transformed_data_mocap = resampled_data_mocap

    # Return the result
    return transformed_data_mocap

Apply synchronization to mocap data.

In [ ]:
data_mocap = sync_data_mocap(
    raw_data_mocap,
    t,
    z_drone,
    do_transform=True,      # <-- whether to align mocap data in space
    t_min_offset=0.,        # <-- how many seconds of mocap data to delete at the start
    t_shift_bnds=[0., 1.],  # <-- range of time shifts over which to search
    t_shift_res=0.05,       # <-- resolution at which to sample time shifts in the range
)

x_mocap = data_mocap['x']
y_mocap = data_mocap['y']
z_mocap = data_mocap['z']
psi_mocap = data_mocap['yaw']
theta_mocap = data_mocap['pitch']
phi_mocap = data_mocap['roll']

Plot all data.

In [ ]:
fig, (ax_x, ax_y, ax_z, ax_psi, ax_theta, ax_phi) = plt.subplots(6, 1, figsize=(8, 8), tight_layout=True)

ax_x.plot(t, x_drone, label='x (drone)')
ax_x.plot(t, x_mocap, '--', label='x (mocap)')
ax_x.legend()
ax_x.grid()

ax_y.plot(t, y_drone, label='y (drone)')
ax_y.plot(t, y_mocap, '--', label='y (mocap)')
ax_y.legend()
ax_y.grid()

ax_z.plot(t, z_drone, label='z (drone)')
ax_z.plot(t, z_mocap, '--', label='z (mocap)')
ax_z.legend()
ax_z.grid()

ax_psi.plot(t, psi_drone, label='yaw (drone)')
ax_psi.plot(t, psi_mocap, '--', label='yaw (mocap)')
ax_psi.legend()
ax_psi.grid()

ax_theta.plot(t, theta_drone, label='pitch (drone)')
ax_theta.plot(t, theta_mocap, '--', label='pitch (mocap)')
ax_theta.legend()
ax_theta.grid()

ax_phi.plot(t, phi_drone, label='roll (drone)')
ax_phi.plot(t, phi_mocap, '--', label='roll (mocap)')
ax_phi.legend()
ax_phi.grid()